<a href="https://colab.research.google.com/github/shahriariit/Violence_Videos_Detection/blob/main/Violence_Detection_from_Hockey_Images_Part_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install moviepy

In [ ]:
from google.colab import drive
import cv2
import os
import math
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score

In [ ]:
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/HDataset'

Mounted at /content/drive


In [ ]:
from google.colab import drive
import cv2
import os
import math
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score
import csv # Import the csv module

def load_images_from_folder(folder, label):
    """Load images from a folder, preprocess, and assign labels."""
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # Load and preprocess image
        img = load_img(img_path, target_size=(224, 224))  # Resize image to 224x224
        img = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)


def build_transfer_learning_model(base_model_fn, input_shape=(224, 224, 3), learning_rate=0.001):
    """
    Builds a binary classification model using a specified base model function.

    Parameters:
    - base_model_fn: a function from keras.applications (e.g., VGG16, ResNet50V2)
    - input_shape: input shape of the images
    - learning_rate: learning rate for the optimizer

    Returns:
    - Compiled Keras Model
    """
    # Load the base model without the top classification layer
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def evaluate(base_model, X_test, y_test, output_file="classification_metrics.csv"):
    # Predict probabilities

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(["Accuracy", "Cohen-Kappa", "Precision", "Recall", "F1-Score", "AUROC", "MCC", "Specificity", "G-Mean", "Type I Error","Type II Error"])

        y_probs = base_model.predict(X_test)
        y_pred = (y_probs > 0.5).astype(int).flatten()

        # Classification Report (includes precision, recall, F1-score)
        print("Classification Report:\n", classification_report(y_test, y_pred))

        # Individual Metrics
        accuracy = accuracy_score(y_test, y_pred)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        auroc = roc_auc_score(y_test, y_probs, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        specificity = specificity_score(y_test, y_pred, average='weighted')
        g_mean = geometric_mean_score(y_test, y_pred, average='weighted')
        type_i_error = 1 - g_mean
        type_ii_error = 1 - specificity


        writer.writerow([accuracy, cohen_kappa, precision, recall, f1, auroc, mcc, specificity, g_mean, type_i_error, type_ii_error])

        # Display results
        print("Accuracy:", accuracy)
        print("Cohen's Kappa:", cohen_kappa)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-Score:", f1)
        print("AUROC:", auroc)
        print("Matthews Correlation Coefficient (MCC):", mcc)
        print("Specificity",specificity)
        print("Geometric Mean",g_mean)
        print("Type I Error",type_i_error)
        print("Type II Error",type_ii_error)

In [ ]:
# Paths to the frames
nonviolence_folder = '/content/drive/MyDrive/HDataset/Nofight'
violence_folder = '/content/drive/MyDrive/HDataset/Fight'

In [ ]:
# Load images and labels for each class
violence_images, violence_labels = load_images_from_folder(violence_folder, 1)  # Label 1 for Violence
nonviolence_images, nonviolence_labels = load_images_from_folder(nonviolence_folder, 0)  # Label 0 for Non-Violence

# Combine the datasets
X = np.concatenate((violence_images, nonviolence_images), axis=0)
y = np.concatenate((violence_labels, nonviolence_labels), axis=0)

# Shuffle the dataset
X, y = shuffle(X, y, random_state=42)

print(f"Dataset loaded and shuffled.")
print(f"Total samples: {len(y)}")
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Dataset loaded and shuffled.
Total samples: 2000
Shape of X: (2000, 224, 224, 3), Shape of y: (2000,)


In [ ]:
# Split dataset into training and testin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_vgg16 = build_transfer_learning_model(VGG16)
model_vgg16.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg16 = model_vgg16.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg16}")
evaluate(model_vgg16, X_test, y_test,"vgg16_classification.csv")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 30s 343ms/step - accuracy: 0.7094 - loss: 0.5612 - val_accuracy: 0.9150 - val_loss: 0.2690
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 196ms/step - accuracy: 0.8994 - loss: 0.2808 - val_accuracy: 0.9100 - val_loss: 0.2469
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 197ms/step - accuracy: 0.9143 - loss: 0.2598 - val_accuracy: 0.9325 - val_loss: 0.2395
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 201ms/step - accuracy: 0.9168 - loss: 0.2464 - val_accuracy: 0.9225 - val_loss: 0.2147
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - accuracy: 0.9313 - loss: 0.2005 - val_accuracy: 0.9375 - val_loss: 0.2108
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 205ms/step - accuracy: 0.9249 - loss: 0.2134 - val_accuracy: 0.9300 - val_loss: 0.2173
Accuracy: 0.9300000071525574
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 224ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.9

In [ ]:
model_vgg19 = build_transfer_learning_model(VGG19)
model_vgg19.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg19 = model_vgg19.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg19}")
evaluate(model_vgg19, X_test, y_test,"vgg19_classification.csv")

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 327ms/step - accuracy: 0.7896 - loss: 0.4693 - val_accuracy: 0.9175 - val_loss: 0.2604
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 311ms/step - accuracy: 0.9006 - loss: 0.2849 - val_accuracy: 0.9100 - val_loss: 0.2583
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 265ms/step - accuracy: 0.9047 - loss: 0.2901 - val_accuracy: 0.9225 - val_loss: 0.2325
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 315ms/step - accuracy: 0.9044 - loss: 0.2611 - val_accuracy: 0.9275 - val_loss: 0.2200
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 21s 321ms/step - accuracy: 0.9064 - loss: 0.2540 - val_accuracy: 0.9225 - val_loss: 0.2466
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 276ms/step - accuracy: 0.9137 - loss: 0.2394 - val_accuracy: 0.9300 - val_loss: 0.2059
Accuracy: 0.9300000071525574
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 254ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.9

In [ ]:
model_mobilenetv3 = build_transfer_learning_model(MobileNetV3Large)
model_mobilenetv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_mobilenetv3 = model_mobilenetv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_mobilenetv3}")
evaluate(model_mobilenetv3, X_test, y_test,"mobilenetv3_classification.csv")

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 26s 285ms/step - accuracy: 0.5783 - loss: 0.6999 - val_accuracy: 0.8125 - val_loss: 0.4866
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - accuracy: 0.7816 - loss: 0.4994 - val_accuracy: 0.7875 - val_loss: 0.4527
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.7816 - loss: 0.4731 - val_accuracy: 0.8375 - val_loss: 0.3845
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8177 - loss: 0.4197 - val_accuracy: 0.8575 - val_loss: 0.3474
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.8301 - loss: 0.3911 - val_accuracy: 0.8625 - val_loss: 0.3452
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8428 - loss: 0.3788 - val_accuracy: 0.8725 - val_loss: 0.3199
Accuracy: 0.8725000023841858
13/13 ━━━━━━━━━━━━━━━━━━━━ 10s 387ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0

In [ ]:
model_resnet50v2 = build_transfer_learning_model(ResNet50V2)
model_resnet50v2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_resnet50v2 = model_resnet50v2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_resnet50v2}")
evaluate(model_resnet50v2, X_test, y_test,"resnet50v2_classification.csv")

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 27s 264ms/step - accuracy: 0.8275 - loss: 0.5463 - val_accuracy: 0.9300 - val_loss: 0.2681
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 25s 100ms/step - accuracy: 0.9403 - loss: 0.1914 - val_accuracy: 0.9375 - val_loss: 0.2023
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 91ms/step - accuracy: 0.9431 - loss: 0.1704 - val_accuracy: 0.9250 - val_loss: 0.2802
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.9550 - loss: 0.1171 - val_accuracy: 0.9375 - val_loss: 0.2249
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.9761 - loss: 0.0721 - val_accuracy: 0.9400 - val_loss: 0.2078
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.9529 - loss: 0.1232 - val_accuracy: 0.9450 - val_loss: 0.1876
Accuracy: 0.9449999928474426
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 338ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      

In [ ]:
model_inceptionresnetv2 = build_transfer_learning_model(InceptionResNetV2)
model_inceptionresnetv2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionresnetv2 = model_inceptionresnetv2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionresnetv2}")
evaluate(model_inceptionresnetv2, X_test, y_test,"inceptionresnetv2_classification.csv")

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 67s 723ms/step - accuracy: 0.7398 - loss: 0.8202 - val_accuracy: 0.8875 - val_loss: 0.3015
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 43s 197ms/step - accuracy: 0.9112 - loss: 0.2557 - val_accuracy: 0.9150 - val_loss: 0.2439
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 200ms/step - accuracy: 0.9177 - loss: 0.2183 - val_accuracy: 0.9125 - val_loss: 0.2335
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 200ms/step - accuracy: 0.9082 - loss: 0.2172 - val_accuracy: 0.9200 - val_loss: 0.2194
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 207ms/step - accuracy: 0.9285 - loss: 0.1770 - val_accuracy: 0.9000 - val_loss: 0.2824
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 205ms/step - accuracy: 0.9107 - loss: 0.2111 - val_accuracy: 0.8925 - val_loss: 0.2754
Accuracy: 0.8924999833106995
13/13 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.8

In [ ]:
model_densenet121 = build_transfer_learning_model(DenseNet121)
model_densenet121.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet121 = model_densenet121.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet121}")
evaluate(model_densenet121, X_test, y_test,"densenet121_classification.csv")

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 58s 583ms/step - accuracy: 0.8750 - loss: 0.3475 - val_accuracy: 0.9225 - val_loss: 0.2264
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 43s 97ms/step - accuracy: 0.9361 - loss: 0.1763 - val_accuracy: 0.9375 - val_loss: 0.2094
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.9549 - loss: 0.1273 - val_accuracy: 0.9350 - val_loss: 0.1889
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 101ms/step - accuracy: 0.9439 - loss: 0.1421 - val_accuracy: 0.9275 - val_loss: 0.2750
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.9616 - loss: 0.1151 - val_accuracy: 0.9275 - val_loss: 0.2115
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.9492 - loss: 0.1222 - val_accuracy: 0.9350 - val_loss: 0.2031
Accuracy: 0.9350000023841858
13/13 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.91      0.9

In [ ]:
model_densenet169 = build_transfer_learning_model(DenseNet169)
model_densenet169.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet169 = model_densenet169.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet169}")
evaluate(model_densenet169, X_test, y_test,"densenet169_classification.csv")

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.8291 - loss: 0.4405 - val_accuracy: 0.9375 - val_loss: 0.1984
Epoch 2/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step - accuracy: 0.9380 - loss: 0.1784 - val_accuracy: 0.9225 - val_loss: 0.2336
Epoch 3/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 120ms/step - accuracy: 0.9501 - loss: 0.1449 - val_accuracy: 0.9400 - val_loss: 0.1861
Epoch 4/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 0.9566 - loss: 0.1397 - val_accuracy: 0.9125 - val_loss: 0.2513
Epoch 5/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.9612 - loss: 0.1106 - val_accuracy: 0.9450 - val_loss: 0.1843
Epoch 6/6
50/50 ━━━━━━━━━━━━━━━━━━━━ 10s 115ms/step - accuracy: 0.9744 - loss: 0.0833 - val_accuracy: 0.9350 - val_loss: 0.1776
Accuracy: 0.9350000023841858
13/13 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0